In [ ]:
#r "nuget:FSharp.Data"
#r "nuget:FSharpX.Collections"

Installed Packages FSharp.Data, 4.2.8 FSharpX.Collections, 3.0.1

In [ ]:
open System
open FSharp.Data
open FSharpx.Collections

let rows = CsvFile.Load("/workspaces/transport tycoon/23/s02e03_test.csv").Rows
rows

index,Columns
0,"[ CARGO_17490, 1859-12-13 10:56:05, Gizbourne, Rustport, 72.17 ]"
1,"[ CARGO_17491, 1859-12-14 03:18:00, Rustport, Gizbourne, 74.55 ]"
2,"[ CARGO_17492, 1859-12-14 15:18:37, Gizbourne, Leverstorm, 72.1 ]"
3,"[ CARGO_17493, 1859-12-15 00:56:21, Leverstorm, Irondale, 65.32 ]"
4,"[ CARGO_17494, 1859-12-15 05:17:14, Irondale, Copperhold, 79.35 ]"
5,"[ CARGO_17495, 1859-12-15 10:33:25, Copperhold, Irondale, 65.47 ]"
6,"[ CARGO_17495, 1859-12-15 17:25:31, Irondale, Gizbourne, 76.58 ]"
7,"[ CARGO_17496, 1859-12-16 00:22:25, Gizbourne, Irondale, 75.7 ]"
8,"[ CARGO_17497, 1859-12-16 06:23:09, Irondale, Gizbourne, 87.49 ]"
9,"[ CARGO_17498, 1859-12-16 13:27:25, Gizbourne, Irondale, 74.39 ]"
